# Ontario sunshine list 2021

The sunshine list is here for 2021, and I put together some graphics for CBC News breaking down what's in it this year.

First, let's import pandas and a few other modules I'll use in the analysis.

In [44]:
import pandas as pd
import numpy as np
import math
import requests
import json

We're going to be looking at a lot of float values, so I'll take the time here to set a global format for them all.

In [45]:
pd.options.display.float_format = '{:,}'.format

Now we read in the data, downloaded [here](https://www.ontario.ca/public-sector-salary-disclosure/2021/all-sectors-and-seconded-employees/).

In [46]:
raw_2021 = pd.read_csv('./raw/2021.csv', encoding="utf-8", encoding_errors='ignore', thousands=',')

Because we'll do a lot of analysis with this raw file, I'm going to make a copy and call it "data".

In [47]:
data = raw_2021.copy()

Then, we'll do a bit of cleaning. We'll create a new column called "name" that is the first and last name combined.

In [48]:
data["Name"] = data["First Name"] + " " + data["Last Name"]

Then, we slice off the French title from the "job title" column, which is usually separated by a slash (forward or back).

In [49]:
data["Job Title"] = data["Job Title"].apply(lambda x: x.split("/|\\")[0].strip())

We're also going to remove the "seconded" sectors, because they don't look all that interesting.

In [50]:
data = (data
        .loc[~data["Sector"].str.contains("Second"), :]
        .sort_values('Salary', ascending=False)
        .set_index("Name")
        )

Next, we're going to bin the data into various earning groups. We do this semi-programmatically by finding max and min values, then using a numpy function to split the range into equal parts.

In [51]:
# Binning into brackets
min_value = data['Salary'].min()
max_value = data['Salary'].max()

bins = np.linspace(min_value,320000,5)

for i, number in enumerate(bins, 0):
    bins[i] = round(number, -3)
    
bins = np.append(bins, round(math.ceil(max_value/100000)*100000, -5))

Now we piece together some nicer labels for each of the bins, and then run the `pd.cut()` method to actually do the binning.

In [52]:
labels = []

for i, number in enumerate(bins, 0):
    try:
        label = "$" + '{:,}'.format(round(bins[i])) + " - $" + '{:,}'.format(round(bins[i+1]))
        labels.append(label)
    except:
        print()
    
data["Earnings Bracket"] = pd.cut(data['Salary'], bins=bins, labels=labels, include_lowest=True).str.replace("00,000", "M").str.replace(",000", "K").str.replace(",", ".").str.replace("1M", "100K")

Now let's take a peek at how it looks.

In [53]:
data.head(5)

,Sector,Last Name,First Name,Salary,Benefits,Employer,Job Title,Year,_docID,Earnings Bracket
Name,,,,,,,,,,
Kenneth Hartwick,Ontario Power Generation,Hartwick,Kenneth,"1,628,246.0","7,539.84",Ontario Power Generation,President and Chief Executive Officer,2021,126612,$320K - $1.7M
Dominique Miniere,Ontario Power Generation,Miniere,Dominique,"1,523,518.84","3,922.56",Ontario Power Generation,Chief Strategy Officer,2021,128750,$320K - $1.7M
Sean Granville,Ontario Power Generation,Granville,Sean,"1,064,429.42","2,632.32",Ontario Power Generation,Chief Operations Officer and Chief Nuclear Off...,2021,126360,$320K - $1.7M
Michael Martelli,Ontario Power Generation,Martelli,Michael,"1,010,834.05","3,758.4",Ontario Power Generation,Chief Projects Officer,2021,128308,$320K - $1.7M
Kevin Smith,Hospitals & Boards of Public Health,Smith,Kevin,"845,092.2","87,153.64",University Health Network,President and Chief Executive Officer,2021,64694,$320K - $1.7M


From here, we'll do a variety of analyses on this now (pretty) clean dataset.

### Employer totals

First, a quick look at the totals for each employer, just to get a sense of the big players here.

In [54]:
data.pivot_table(index="Employer", values="Salary", aggfunc="sum").sort_values("Salary", ascending=False)

,Salary
Employer,
Ontario Power Generation,"1,284,854,409.88"
Toronto District School Board,"1,125,152,992.1"
City Of Toronto,"914,069,833.63"
University Of Toronto,"806,686,517.09"
Ontario Provincial Police,"701,360,163.01"
...,...
Pwi-di-goo-zing-ne-yaa-zhing Advisory Services,"100,073.0"
Across Boundaries,"100,001.23"
Wesley Clover Foundation,"100,000.0"


### Binning data and breakdowns by industry

In [55]:
bracket_counts = pd.pivot_table(data, index="Earnings Bracket", columns="Sector", aggfunc='count', values="Year")

for label, col in bracket_counts.items():
    bracket_counts[label] = round((bracket_counts[label] / bracket_counts[label].sum()) * 100, 2)
    
bracket_counts

Sector,Colleges,Crown Agencies,Government of Ontario – Judiciary,Government of Ontario – Legislative Assembly and Offices,Government of Ontario – Ministries,Hospitals & Boards of Public Health,Municipalities & Services,Ontario Power Generation,Other Public Sector Employers,School Boards,Universities
Earnings Bracket,,,,,,,,,,,
$100K - $155K,94.27,83.61,43.97,78.3,86.07,91.61,92.05,63.98,83.79,99.18,55.25
$155K - $210K,4.81,11.76,6.19,15.57,8.15,4.94,7.05,27.62,10.11,0.73,33.65
$210K - $265K,0.6,2.98,1.9,5.19,5.27,1.26,0.7,6.17,3.25,0.07,8.19
$265K - $320K,0.24,0.94,3.81,0.47,0.23,0.81,0.14,1.15,2.05,0.01,1.88
$320K - $1.7M,0.09,0.71,44.13,0.47,0.28,1.38,0.06,1.08,0.79,0.0,1.03


In [56]:
universities = pd.pivot_table(data.loc[data["Sector"] == "Universities", :], index="Earnings Bracket", columns="Employer", aggfunc='count', values="Year")

for label, col in universities.items():
    universities[label] = round((universities[label] / universities[label].sum()) * 100, 2)
    


In [57]:
ministries = pd.pivot_table(data.loc[data["Sector"] == "Government of Ontario – Ministries", :], index="Earnings Bracket", columns="Employer", aggfunc='count', values="Year")

for label, col in ministries.items():
    ministries[label] = round((ministries[label] / ministries[label].sum()) * 100, 2)
    
ministries.to_clipboard()

In [58]:
municipalities = pd.pivot_table(data.loc[data["Sector"] == "Crown Agencies", :], index="Earnings Bracket", columns="Employer", aggfunc='count', values="Year")

for label, col in municipalities.items():
    municipalities[label] = round((municipalities[label] / municipalities[label].sum()) * 100, 2)
    
municipalities.transpose().sort_values("$100K - $155K", ascending=True).head(20).to_clipboard()

### Top 10 earners

In [59]:
data.loc[:, "Name/Title"] = "**" + data.index + "**<br>" + data["Job Title"]

df_tops = (data
           .pivot_table(columns="Year", values="Salary", index=["Name/Title", "Employer"], aggfunc="min")
           .sort_values(2021, ascending=False)
           .head(50)
           )

df_tops.to_clipboard()

### Ranges

In [60]:
data

,Sector,Last Name,First Name,Salary,Benefits,Employer,Job Title,Year,_docID,Earnings Bracket,Name/Title
Name,,,,,,,,,,,
Kenneth Hartwick,Ontario Power Generation,Hartwick,Kenneth,"1,628,246.0","7,539.84",Ontario Power Generation,President and Chief Executive Officer,2021,126612,$320K - $1.7M,**Kenneth Hartwick**<br>President and Chief Ex...
Dominique Miniere,Ontario Power Generation,Miniere,Dominique,"1,523,518.84","3,922.56",Ontario Power Generation,Chief Strategy Officer,2021,128750,$320K - $1.7M,**Dominique Miniere**<br>Chief Strategy Officer
Sean Granville,Ontario Power Generation,Granville,Sean,"1,064,429.42","2,632.32",Ontario Power Generation,Chief Operations Officer and Chief Nuclear Off...,2021,126360,$320K - $1.7M,**Sean Granville**<br>Chief Operations Officer...
Michael Martelli,Ontario Power Generation,Martelli,Michael,"1,010,834.05","3,758.4",Ontario Power Generation,Chief Projects Officer,2021,128308,$320K - $1.7M,**Michael Martelli**<br>Chief Projects Officer
Kevin Smith,Hospitals & Boards of Public Health,Smith,Kevin,"845,092.2","87,153.64",University Health Network,President and Chief Executive Officer,2021,64694,$320K - $1.7M,**Kevin Smith**<br>President and Chief Executi...
...,...,...,...,...,...,...,...,...,...,...,...
Nathalie Mouralian,Municipalities & Services,Mouralian,Nathalie,"100,000.0",0.0,City Of Toronto,Support Assistant C,2021,104078,$100K - $155K,**Nathalie Mouralian**<br>Support Assistant C
Tony Dunn,Other Public Sector Employers,Dunn,Tony,"100,000.0","23,297.82",Wesley Clover Foundation,Chief Operations Officer,2021,134132,$100K - $155K,**Tony Dunn**<br>Chief Operations Officer
Roderick Brereton,Other Public Sector Employers,Brereton,Roderick,"100,000.0",0.0,Urban Rez Solutions Social Enterprise,Executive Director,2021,132871,$100K - $155K,**Roderick Brereton**<br>Executive Director


In [61]:
sectors = data.groupby("Sector").count().loc[:, ["Last Name"]].sort_values("Last Name", ascending=False)

sectors

,Last Name
Sector,
School Boards,80434
Municipalities & Services,53835
Hospitals & Boards of Public Health,30571
Universities,23096
Government of Ontario – Ministries,20319
Crown Agencies,9824
Other Public Sector Employers,8731
Ontario Power Generation,8520
Colleges,7864


In [62]:
sector_max = data.pivot_table(index="Sector", values="Salary", aggfunc="max")
sector_avg = data.pivot_table(index="Sector", values="Salary", aggfunc="median")
sector_total = data.pivot_table(index="Sector", values="Salary", aggfunc="sum")

sectors = pd.concat([sector_max, sector_avg, sector_total], axis=1)
sectors.columns = ["Max", "Average", "Total"]

sectors = sectors.sort_values("Total", ascending=False)

sectors.to_clipboard()

### Carleton University

In [63]:
data.loc[data["Employer"] == "Carleton University", :].sort_values("Salary", ascending=False).to_clipboard()

### Split by industry into different excel sheets

In [64]:
# categories = data_2020["Sector"].unique()
# sheets = []

# for category in categories:
#     df = data_2020.loc[data_2020["Sector"] == category, :]
#     df.loc[:, "Name/Title"] = "**" + df["Name"] + "**<br>" + df["Job Title"]
#     df = df.loc[:, ["Name/Title", "Employer", "Salary", "Benefits"]]
#     sheets.append(df)

In [65]:
# writer = pd.ExcelWriter('exports/sunshine_list.xlsx')

# for i, df in enumerate(sheets, 0):
#     df.to_excel(writer, index=False,sheet_name=f'{categories[i]}'[:30])

# writer.save()

### Gender analysis

In [66]:
url = 'https://parseapi.back4app.com/classes/Complete_List_Names?limit=500000'
headers = {
    'X-Parse-Application-Id': 'zsSkPsDYTc2hmphLjjs9hz2Q3EXmnSxUyXnouj1I',
    'X-Parse-Master-Key': '4LuCXgPPXXO2sU5cXm6WwpwzaKyZpo3Wpj4G4xXK'
}
name_data = requests.get(url, headers=headers).json()["results"]
data2 = json.dumps(name_data)
names = pd.read_json(data2)

display(names)

,objectId,Name,Gender,createdAt,updatedAt
0,lSxg9sIUv9,Will,male,2020-01-23T23:31:09.261Z,2020-01-23T23:31:09.261Z
1,Ypp4vpokki,James,male,2020-01-23T23:31:09.241Z,2020-01-23T23:31:09.241Z
2,tZYFQ1Yjyj,Samuel,male,2020-01-23T23:31:09.260Z,2020-01-23T23:31:09.260Z
3,LxQmByBQ1y,John,male,2020-01-23T23:31:09.240Z,2020-01-23T23:31:09.240Z
4,2S3LArF0dw,George,male,2020-01-23T23:31:09.241Z,2020-01-23T23:31:09.241Z
...,...,...,...,...,...
257995,kAzsJnXLBn,Diya,female,2020-01-23T23:43:40.706Z,2020-01-23T23:43:40.706Z
257996,1Do0BIVl3n,Kenley,female,2020-01-23T23:43:40.706Z,2020-01-23T23:43:40.706Z
257997,X5boZIpAnZ,Elianna,female,2020-01-23T23:43:40.706Z,2020-01-23T23:43:40.706Z
257998,u6izCGBJUz,Iyana,female,2020-01-23T23:43:40.706Z,2020-01-23T23:43:40.706Z


In [69]:
name_count = (names
              .value_counts(subset=["Name","Gender"])
              .reset_index()
              .pivot(columns="Gender", index="Name")
              .fillna(0)
              )

name_count.columns = name_count.columns.droplevel(0)

name_count["chance_male"] = name_count["male"] / (name_count["male"] + name_count["female"])
name_count["Gender"] = np.where(name_count['chance_male']>=0.65, 'Male', np.where(name_count['chance_male']<=0.35, 'Female', pd.NA))

gender_list = name_count[["Gender"]].reset_index()

display(gender_list)

Gender,Name,Gender
0,Aaden,Male
1,Aaliyah,Female
2,Aarav,Male
3,Aaron,Male
4,Ab,Male
...,...,...
6777,Zula,Female
6778,malece,Male
6779,maled,Male
6780,malesie,Male


In [98]:
with_gender = data.merge(gender_list, left_on='First Name', right_on="Name", how="left").drop_duplicates()

with_gender = with_gender[["Name/Title", "Sector", "Job Title", "Employer", "Salary", "Benefits", "Earnings Bracket", "Gender"]]
with_gender["Gender"] = with_gender["Gender"].fillna("Unknown")

with_gender.head(3)

,Name/Title,Sector,Job Title,Employer,Salary,Benefits,Earnings Bracket,Gender
0,**Kenneth Hartwick**<br>President and Chief Ex...,Ontario Power Generation,President and Chief Executive Officer,Ontario Power Generation,"1,628,246.0","7,539.84",$320K - $1.7M,Male
1,**Dominique Miniere**<br>Chief Strategy Officer,Ontario Power Generation,Chief Strategy Officer,Ontario Power Generation,"1,523,518.84","3,922.56",$320K - $1.7M,Unknown
2,**Sean Granville**<br>Chief Operations Officer...,Ontario Power Generation,Chief Operations Officer and Chief Nuclear Off...,Ontario Power Generation,"1,064,429.42","2,632.32",$320K - $1.7M,Male


In [107]:
avg_salary = round(with_gender.pivot_table(index="Gender", values="Salary", aggfunc="mean"),0)
count = round(with_gender.pivot_table(index="Gender", values="Name/Title", aggfunc="count"),0)
sum = round(with_gender.pivot_table(index="Gender", values="Salary", aggfunc="sum"),0)
median = round(with_gender.pivot_table(index="Gender", values="Salary", aggfunc="median"),0)

pd.concat([avg_salary, count, sum, median], axis=1)

,Salary,Name/Title,Salary,Salary
Gender,,,,
Female,"117,920.0",91098,"10,742,236,236.0","106,625.0"
Male,"128,700.0",77276,"9,945,439,904.0","116,667.0"
Unknown,"125,673.0",75874,"9,535,344,526.0","112,721.0"
